Задача -  используя датасет Chest X-Ray Images (Pneumonia), необходимо обучить модель на обнаружение болезни, а также реализовать интерфейс с возможностью загрузки и обработки пользовательских снимков


In [ ]:
!pip install gradio

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import gradio as gr
from PIL import Image
import gdown

In [ ]:
gdown.download('https://drive.google.com/uc?export=download&id=15xVUr7waUu7fwDKErsIZQDRCvml1FX86', None, quiet=True)

'Chest X-Ray Images (Pneumonia).zip'

In [ ]:
!unzip -qo "Chest X-Ray Images (Pneumonia)"

In [ ]:
dataset_dir = "/content/chest_xray"

In [ ]:
# Размер изображений и параметры обучения
img_size = (224, 224)
batch_size = 32
epochs = 10
learning_rate = 0.0001

In [ ]:
# Генератор данных с меньшей аугментацией
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_data = train_datagen.flow_from_directory(
    os.path.join(dataset_dir, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary"
)
val_data = val_test_datagen.flow_from_directory(
    os.path.join(dataset_dir, "val"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary"
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
test_data = val_test_datagen.flow_from_directory(
    os.path.join(dataset_dir, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [ ]:
# Упрощенная модель с MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(img_size[0], img_size[1], 3)
)
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
# Обучение модели
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 112s 580ms/step - accuracy: 0.8641 - loss: 0.3020 - val_accuracy: 0.6875 - val_loss: 0.5068
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 528ms/step - accuracy: 0.9462 - loss: 0.1278 - val_accuracy: 0.8750 - val_loss: 0.2766
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 89s 524ms/step - accuracy: 0.9552 - loss: 0.1137 - val_accuracy: 0.8750 - val_loss: 0.2047
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 529ms/step - accuracy: 0.9609 - loss: 0.1055 - val_accuracy: 0.8125 - val_loss: 0.4046
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 524ms/step - accuracy: 0.9647 - loss: 0.0919 - val_accuracy: 0.8750 - val_loss: 0.2145
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 528ms/step - accuracy: 0.9622 - loss: 0.0958 - val_accuracy: 0.8750 - val_loss: 0.2028
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 531ms/step - accuracy: 0.9632 - loss: 0.1038 - val_accuracy: 0.8750 - val_loss: 0.1793
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 93s 547ms/step - accuracy: 0.9673 - loss: 0.0871 - va

In [ ]:
# Оценка на тестовых данных
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Точность на тестовых данных: {test_accuracy * 100:.2f}%")

20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 192ms/step - accuracy: 0.8583 - loss: 0.3813
Точность на тестовых данных: 89.26%


In [ ]:
# Функция для предсказания
def predict_diagnosis(image):
    """
    Предсказать диагноз на основе загруженного изображения.
    """
    # Преобразование изображения
    image = image.resize(img_size)
    image = np.array(image) / 255.0  # Нормализация
    image = np.expand_dims(image, axis=0)  # Добавляем размерность для batch

    # Предсказание
    prediction = model.predict(image)
    if prediction[0][0] > 0.5:
        return "Диагноз: Пневмония", f"Вероятность: {prediction[0][0]:.2f}"
    else:
        return "Диагноз: Норма", f"Вероятность: {1 - prediction[0][0]:.2f}"

# Интерфейс Gradio
interface = gr.Interface(
    fn=predict_diagnosis,
    inputs=gr.Image(type="pil", label="Загрузите рентгеновский снимок"),
    outputs=[
        gr.Textbox(label="Результат"),
        gr.Textbox(label="Вероятность")
    ],
    title="Диагностика Пневмонии",
    description="Загрузите рентгеновский снимок легких, чтобы получить диагноз (норма или пневмония)."
)

# Запуск интерфейса
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bd7a821f48923d184a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
